In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

print('hello')
# 버전 확인
print(f"pandas 버전: {pd.__version__}")
print(f"numpy 버전: {np.__version__}")


In [ ]:

# # 리스트로 Series 생성
# sales = pd.Series([120, 150, 180, 200, 160])
# print("매출 데이터:")
# print(sales)
# print(f"\n타입: {type(sales)}")

# # NumPy 배열
# numpy_array = np.array([10, 20, 30, 40, 50])
# print("\nNumPy 배열:", numpy_array)
# print("타입:", type(numpy_array))
# print("데이터 타입:", numpy_array.dtype)

# # Plotly
# fig = go.Figure()
# fig.show()

In [ ]:
#현재 작업 경로 확인
import os
print(os.getcwd())

#파일 불러오기
df = pd.read_csv('rd/union1.csv', encoding='utf-8')
print("cp949로 로드 성공!")

In [ ]:
#데이터 살펴보기

df.head()
df.info()
df.describe()
df.shape
df.columns.tolist()

⭐목표 정의

계약일의 흐름에 따라 부동산 실거래가가 어떻게 변하는지 자치구명, 법정동명 차원에서 분석

In [ ]:
#계약일을 datetime형식으로 변환
df['계약일'] = pd.to_datetime(df['계약일'], format= '%Y%m%d')
df['물건금액'] = round(df['물건금액'], 0)

print(df['계약일'].head())
print(df['물건금액'].head())

In [ ]:

#자치구-법정동 기준
trend_detail = df.groupby(['계약일','자치구명','법정동명'])['물건금액'].mean().round(0).unstack(level=['자치구명','법정동명']).sort_index(axis=1)

#자치구별 기준
trend_gu = trend_detail.groupby(level='자치구명', axis=1).mean().round(0)

#전체 강남3구 기준
trend_total = df.groupby(['계약일'])['물건금액'].mean().round(0)  
overall_avg = trend_total.mean().mean()

print(f"전체 강남3구 기준 평균거래금액: {overall_avg:,.2f}만 원")

print(trend_detail.head())
print(trend_total.head())

In [ ]:
# 인덱스를 datetime으로 변경⭐⭐⭐ 
# groupby하면서 '계약일'이 인덱스가 되면서 datetime이 풀렸을 가능성이 큼
trend_detail.index = pd.to_datetime(trend_detail.index)
trend_gu.index = pd.to_datetime(trend_gu.index)
trend_total.index = pd.to_datetime(trend_total.index)

#구-동 일별 데이터 ==> 월별로 resampling하기
trend_detail_monthly = trend_detail.resample('MS').mean()

# 구별 데이터를 일별 ==> 월별로 resampling하기
trend_gu_monthly = trend_gu.resample('MS').mean()

# 강남3구 데이터를 일별 ==> 월별로 resampling하기
trend_total_monthly = trend_total.resample('MS').mean()

print(trend_detail_monthly.head())
print(trend_gu_monthly.head())
print(trend_total_monthly.head())

print(f"시구별 데이터 행 개수: {len(trend_detail_monthly)}")
print(f"구별 데이터 행 개수: {len(trend_gu_monthly)}")
print(f"강남3구 전체 평균 행 개수: {len(trend_total_monthly)}")


In [ ]:
# # 데이터가 없는 달(NaN) 제거
# trend_detail_monthly = trend_detail_monthly.dropna()
# trend_gu_monthly = trend_gu_monthly.dropna()
# trend_total_monthly = trend_total_monthly.dropna()

# print(trend_detail_monthly.head())

In [ ]:
print(f"전체 행 개수: {len(trend_total)}")
print(f"전체 행 개수: {len(plot_total)}")

In [ ]:
print(f"전체 행 개수: {len(plot_total)}")

In [ ]:
### 메인 그래프 함수로 재사용 가능하게 변수에 저장 ###

import plotly.graph_objects as go # 시각화 도구를 가져옵니다.
import pandas as pd # 데이터 처리를 위해 판다스를 가져옵니다.

def draw_trend_graph(data): # [추가] 전체 코드를 'draw_trend_graph'라는 이름의 변수(함수)로 지정합니다.
    # 1. 데이터 복사 및 타임스탬프 변환
    plot_gu = data.copy() # 입력받은 데이터를 복사하여 사용합니다.
    if hasattr(plot_gu.index, 'to_timestamp'):
        plot_gu.index = plot_gu.index.to_timestamp()

    # 2. 강남 3구 통합 데이터 계산
    targets = ['강남구', '송파구', '서초구']
    if '강남3구_전체' not in plot_gu.columns:
        plot_gu['강남3구_전체'] = plot_gu[targets].mean(axis=1)

    # 3. 단일 도표 객체 생성
    fig = go.Figure()

    # 4. 각 구별 꺾은선 추가
    colors = {'강남구': 'royalblue', '송파구': 'forestgreen', '서초구': 'darkorange'}
    for gu in targets:
        fig.add_trace(go.Scatter(
            x=plot_gu.index,
            y=plot_gu[gu],
            mode='lines',
            name=gu,
            line=dict(color=colors[gu], width=2),
            hovertemplate='%{x|%Y-%m}<br>' + gu + ': %{y:,.0f}만 원<extra></extra>'
        ))

    # 5. 강남 3구 전체 평균 선 추가 (강조)
    fig.add_trace(go.Scatter(
        x=plot_gu.index,
        y=plot_gu['강남3구_전체'],
        mode='lines',
        name='<b>⭐ 강남 3구 전체 평균</b>',
        line=dict(color='firebrick', width=4),
        hovertemplate='<b>%{x|%Y-%m} 통합 평균: %{y:,.0f}만 원</b><extra></extra>'
    ))

    # 6. 전체 디자인 및 X축 날짜 설정
    fig.update_layout(
        title=dict(
            text='<b>강남 3구(강남·송파·서초) 및 통합 월별 가격 추이 비교</b>',
            x=0.5, font=dict(size=22)
        ),
        width=1200, height=700,
        xaxis=dict(title=dict(text='계약월', standoff=30)),
        yaxis_title='평균 거래가 (만 원)',
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5
        ),
    )

    # X축 설정 업데이트
    start_date = plot_gu.index.min()
    end_date = plot_gu.index.max()
    fig.update_xaxes(
        range=[start_date, end_date],
        tickvals=plot_gu.index,
        ticktext=[f"{m.month}월" for m in plot_gu.index],
        tickformat="%m월", 
        dtick="M1", 
        tickangle=0,
    )

    # 연도 구분선 추가
    years = [2024, 2025, 2026]
    for year in years:
        year_start = f"{year}-01-01"
        if plot_gu.index.min() <= pd.to_datetime(year_start) <= plot_gu.index.max():
            fig.add_vline(
                x=year_start,
                line_width=1.5,
                line_dash="solid",
                line_color="rgba(200, 200, 200, 0.5)"
            )
            fig.add_annotation(
                x=year_start, y=1, yref="paper",
                text=f"<b>{year}년</b>",
                showarrow=False, font=dict(color="gray", size=12),
                xanchor="center", yanchor="bottom"
            )

    # Y축 숫자 콤마 설정
    fig.update_yaxes(tickformat=',d')

    return fig # [추가] 완성된 그래프 객체를 결과값으로 돌려줍니다.

# --- 실제 사용 예시 ---
# 1. 함수를 실행하고 그 결과를 'my_chart'라는 변수에 저장합니다.
my_chart = draw_trend_graph(trend_gu_monthly) 

# 2. 저장된 변수를 호출하여 그래프를 보여줍니다.
my_chart.show()

In [ ]:
# 2025년 강남3구 개별 및 전체 월별 가격 추이

# 1. trend_gu_monthly 데이터에서 2025년 데이터만 쏙 뽑아옵니다.
data_2025 = trend_gu_monthly.loc['2025'] 

# 2. 함수를 실행할 때 위에서 뽑은 '2025년 데이터'를 넣어줍니다.
my_chart = draw_trend_graph(data_2025) 

# 3. 저장된 변수를 호출하여 그래프를 보여줍니다.
my_chart.show()

## 왜 송파구만 저렇게 거래가가 낮지?

### 송파구 vs (강남구, 서초구) 비교해보자! 

In [19]:
# # 1. 대상 구(강남, 송파, 서초) 필터링
# targets = ['강남구', '송파구', '서초구']
# gu_comparison = df[df['자치구명'].isin(targets)].copy()

# 2. 구별 거래 건수(Count), 평균 금액(Mean), 합계 금액(Sum) 비교
# MECE하게 분석하기 위해 합계까지 포함하여 비교합니다.
comparison_stats = df.groupby('자치구명').agg(
    거래건수=('물건금액', 'count'),      # 얼마나 자주 거래되었나 (Volume)
    평균가=('물건금액', 'mean'),        # 전형적인 가격 수준 (Average)
    합계금액=('물건금액', 'sum')         # 해당 지역 전체 자금 규모 (Total Amount)
).round(0)

# 3. 거래 건수 순으로 정렬하여 보기
print("강남 3구 거래 규모 및 가격 비교")
print(comparison_stats.sort_values(by='거래건수', ascending=False))

강남 3구 거래 규모 및 가격 비교
       거래건수       평균가        합계금액
자치구명                             
송파구   12306  173686.0  2137376727
강남구   10075  256579.0  2585036895
서초구    7331  260071.0  1906578545


In [21]:
import plotly.graph_objects as go

# 1. 전년 동월 대비 변동률(Year-over-Year) 계산
# .pct_change(12)는 12칸 전(1년 전) 데이터와의 변화율을 계산합니다.
trend_yoy = trend_gu_monthly.pct_change(12) * 100 

# 계산을 위해 2025년 데이터를 포함한 전체 기간 사용 (2024년 데이터가 있어야 비교 가능)
plot_yoy = trend_yoy.dropna().copy()
if hasattr(plot_yoy.index, 'to_timestamp'):
    plot_yoy.index = plot_yoy.index.to_timestamp()

# 2. 그래프 생성
fig_yoy = go.Figure()

colors = {'강남구': 'royalblue', '송파구': 'forestgreen', '서초구': 'darkorange'}
targets = ['강남구', '송파구', '서초구']

for gu in targets:
    fig_yoy.add_trace(go.Scatter(
        x=plot_yoy.index,
        y=plot_yoy[gu],
        mode='lines+markers',
        name=f"{gu} 변동률",
        line=dict(color=colors[gu], width=2),
        hovertemplate='%{x|%Y-%m}<br>' + gu + ' 변동률: %{y:.2f}%<extra></extra>'
    ))

# 3. 레이아웃 설정 (0% 강조선 추가)
fig_yoy.update_layout(
    title=dict(text='<b>강남 3구 전년 동월 대비 가격 변동률 (YoY %)</b>', x=0.5),
    xaxis_title='계약월',
    yaxis_title='변동률 (%)',
    template='plotly_white',
    hovermode='x unified'
)

# 0% 기준선 추가 (상승/하락 구분용)
fig_yoy.add_hline(y=0, line_dash="dash", line_color="black", line_width=1)

# X축 날짜 형식 설정
fig_yoy.update_xaxes(tickformat="%y년 %m월", dtick="M3")

fig_yoy.show()

In [23]:
import pandas as pd
import plotly.graph_objects as go

# 1. 전년 동월 대비 변동률(YoY) 계산
# .pct_change(12)를 통해 작년 같은 달과 비교한 변화율을 구합니다.
trend_yoy = (trend_gu_monthly.pct_change(12) * 100).dropna()

# 2. 시각화를 위한 데이터 재구성 함수
def get_yearly_comparison_data(yoy_data, gu_name):
    # 인덱스에서 연도와 월을 추출합니다.
    temp_df = yoy_data[[gu_name]].copy()
    temp_df['Year'] = temp_df.index.year
    temp_df['Month'] = temp_df.index.month
    return temp_df

# 3. 그래프를 그리는 메인 로직
targets = ['강남구', '서초구', '송파구']
colors = {'2024': 'rgba(100, 100, 100, 0.5)', '2025': 'firebrick'} # 2024년은 연하게, 2025년은 진하게

for gu in targets:
    # 해당 구의 데이터를 연도/월별로 정리
    df_plot = get_yearly_comparison_data(trend_yoy, gu)
    
    fig = go.Figure()
    
    # 2024년과 2025년 선을 각각 추가
    for year in [2024, 2025]:
        year_data = df_plot[df_plot['Year'] == year]
        
        # 데이터가 있는 경우에만 그래프 생성
        if not year_data.empty:
            fig.add_trace(go.Scatter(
                x=year_data['Month'],
                y=year_data[gu],
                mode='lines+markers',
                name=f"{year}년",
                line=dict(width=4 if year == 2025 else 2, color=colors[str(year)]),
                hovertemplate='%{x}월 변동률: %{y:.2f}%<extra></extra>'
            ))

    # 4. 그래프 디자인 설정
    fig.update_layout(
        title=dict(text=f'<b>[{gu}] 전년 동월 대비 가격 변동률 비교 (2024, 2025)</b>', x=0.5),
        xaxis=dict(
            title='계약월',
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=[f'{m}월' for m in range(1, 13)]
        ),
        yaxis_title='전년 대비 변동률 (%)',
        template='plotly_white',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        hovermode='x unified'
    )
    
    # 0% 기준선 추가
    fig.add_hline(y=0, line_dash="solid", line_color="black", line_width=1)
    
    fig.show()

In [24]:
import pandas as pd
import plotly.graph_objects as go

# 1. 강남 3구 통합 월별 데이터 생성 (가로 방향 평균)
targets = ['강남구', '서초구', '송파구']
trend_gu_monthly['강남3구_전체'] = trend_gu_monthly[targets].mean(axis=1)

# 2. 전년 동월 대비 변동률(YoY) 계산
# 12개월 전과 비교하여 변동 폭을 퍼센트로 나타냅니다.
trend_total_yoy = (trend_gu_monthly['강남3구_전체'].pct_change(12) * 100).dropna()

# 3. 시각화를 위한 데이터 재구성 (연도/월 분리)
df_total_plot = pd.DataFrame(trend_total_yoy)
df_total_plot['Year'] = df_total_plot.index.year
df_total_plot['Month'] = df_total_plot.index.month

# 4. 그래프 생성
fig = go.Figure()

# 연도별 설정 (2024년은 배경색, 2025년은 강조색)
colors = {'2024': 'rgba(150, 150, 150, 0.5)', '2025': 'firebrick'}

for year in [2024, 2025]:
    year_data = df_total_plot[df_total_plot['Year'] == year]
    
    if not year_data.empty:
        fig.add_trace(go.Scatter(
            x=year_data['Month'],
            y=year_data['강남3구_전체'],
            mode='lines+markers',
            name=f"{year}년",
            # 2025년 선을 더 굵게 하여 주인공임을 강조합니다.
            line=dict(width=4 if year == 2025 else 2, color=colors[str(year)]),
            marker=dict(size=8 if year == 2025 else 6),
            hovertemplate='%{x}월 변동률: %{y:.2f}%<extra></extra>'
        ))

# 5. 디자인 및 레이아웃 설정
fig.update_layout(
    title=dict(text='<b>[강남 3구 전체] 전년 동월 대비 가격 변동률 비교 (2024 vs 2025)</b>', x=0.5),
    xaxis=dict(
        title='계약월',
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=[f'{m}월' for m in range(1, 13)]
    ),
    yaxis_title='전년 대비 변동률 (%)',
    template='plotly_white',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode='x unified'
)

# 0% 기준선 추가 (상승과 하락의 경계)
fig.add_hline(y=0, line_dash="solid", line_color="black", line_width=1.5)

fig.show()

### 송파 vs (강남, 서초) 비교 데이터 시각화 

In [25]:
import plotly.express as px # 시각화 라이브러리를 가져옵니다.

# 1. 인덱스 변환
plot_stats = comparison_stats.reset_index() # 인덱스를 컬럼으로 바꿉니다.

# 2. 거래건수 시각화 (시장 활성도)
fig_count = px.bar(
    plot_stats, 
    x='자치구명', 
    y='거래건수', 
    text_auto=True, # 숫자를 막대 위에 표시합니다.
    title='<b>강남 3구 지역별 거래건수 (Volume)</b>', # 제목을 설정합니다.
    color='자치구명', # 구별로 색상을 나눕니다.
    template='plotly_white' # 흰색 배경을 사용합니다.
)

# [수정] 거래건수 그래프 레이아웃 최적화
fig_count.update_layout(
    width=600, # 너비를 600으로 줄입니다.
    height=450, # 높이를 450으로 줄입니다.
    title=dict(font=dict(size=20), x=0.5), # 제목 크기를 20으로 줄이고 중앙에 둡니다.
    xaxis=dict(title="", tickfont=dict(size=14)), # X축 제목을 없애고 글자 크기를 조정합니다.
    yaxis=dict(title="거래건수 (건)", tickfont=dict(size=12), tickformat=",d"), # Y축 숫자 포맷을 설정합니다.
    showlegend=False, # 범례를 숨깁니다.
    margin=dict(l=60, r=20, t=80, b=40), # 여백을 줄여 막대 영역을 넓힙니다.
    bargap=0.4 # 막대 사이의 간격을 적절히 벌립니다.
)
fig_count.show() # 그래프를 출력합니다.

# 3. 평균가 시각화 (가격 수준)
fig_mean = px.bar(
    plot_stats, 
    x='자치구명', 
    y='평균가', 
    text_auto=',.0f', # 천 단위 콤마를 표시합니다.
    title='<b>강남 3구 지역별 평균 거래가 (Average Price)</b>', # 제목을 설정합니다.
    color='자치구명', # 색상을 지정합니다.
    template='plotly_white' # 배경 스타일입니다.
)

# [수정] 평균가 그래프 레이아웃 최적화
fig_mean.update_layout(
    width=600, # 너비를 600으로 줄입니다.
    height=450, # 높이를 450으로 줄입니다.
    title=dict(font=dict(size=20), x=0.5), # 제목 크기를 20으로 줄이고 중앙 배치합니다.
    xaxis=dict(title="", tickfont=dict(size=14)), # X축 설정을 조정합니다.
    yaxis=dict(title="평균가 (만 원)", tickfont=dict(size=12), tickformat=",d"), # Y축 숫자 포맷을 설정합니다.
    showlegend=False, # 범례를 숨깁니다.
    margin=dict(l=60, r=20, t=80, b=40), # 여백을 최적화합니다.
    bargap=0.4 # 막대 두께를 조절합니다.
)
fig_mean.show() # 그래프를 출력합니다.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. 2025년 데이터 필터링 및 집계
df_2025 = df_target[df_target['계약일'].dt.year == 2025]
stats_2025 = df_2025.groupby('자치구명')['물건금액'].agg(['count', 'mean']).reset_index()
stats_2025.columns = ['자치구명', '거래건수', '평균가']

# 2. 이중 축 도표 생성
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 3. 주축(왼쪽): 평균 거래가 (막대 그래프)
# 막대 너비를 줄이기 위해 marker_line_width 등을 활용하거나 bargap을 조정합니다.
fig.add_trace(
    go.Bar(
        x=stats_2025['자치구명'], 
        y=stats_2025['평균가'],
        name='평균 거래가 (Price)',
        marker_color='rgba(100, 149, 237, 0.7)', # 부드러운 파란색
        text=stats_2025['평균가'].apply(lambda x: f"{x:,.0f}"),
        textposition='auto',
    ),
    secondary_y=False, # 왼쪽 주축 사용
)

# 4. 보조축(오른쪽): 거래건수 (꺾은선 그래프)
fig.add_trace(
    go.Scatter(
        x=stats_2025['자치구명'], 
        y=stats_2025['거래건수'],
        name='거래건수 (Volume)',
        mode='lines+markers+text',
        line=dict(color='firebrick', width=3),
        marker=dict(size=10),
        text=stats_2025['거래건수'],
        textposition='top center',
    ),
    secondary_y=True, # 오른쪽 보조축 사용
)

# 5. 레이아웃 및 디자인 설정
fig.update_layout(
    title=dict(
        text='<b>2025년 강남 3구 지역별 평균가 및 거래건수 비교</b>',
        x=0.5, font=dict(size=22)
    ),
    template='plotly_white',
    # 범례를 아래쪽 중앙으로 이동
    legend=dict(
        orientation="h",        # 가로 방향 배치
        yanchor="top",          # 기준점 위쪽
        y=-0.2,                 # 그래프 아래 여백으로 이동
        xanchor="center",       # 가로 기준점 중앙
        x=0.5                   # 중앙 위치
    ),
    width=800, height=600,
    margin=dict(l=50, r=50, t=100, b=100),
    # 막대 너비를 현재의 2/3 정도로 줄이기 위해 bargap을 0.5 수준으로 높임
    bargap=0.5 
)

# 축 이름 및 숫자 포맷 설정
fig.update_yaxes(title_text="평균 거래가 (만 원)", secondary_y=False, tickformat=",d")
fig.update_yaxes(title_text="거래건수 (건)", secondary_y=True, tickformat=",d")

fig.show()

_IncompleteInputError: incomplete input (2592968161.py, line 63)

#### 아! 거래총액이 2위인걸로 봐서 평균가격이 낮은 이유는 거래 건수는 많은데 거래가가 낮아서 그렇구나! 
####그럼 이 거래가가 왜 낮지? 혹시 평형수가 작은 아파트들이 강남, 서초에 비해 많은가? 

In [ ]:
# 1. 현재 df가 가진 모든 컬럼명을 한 줄로 출력합니다.
print("실제 컬럼명 리스트:", df.columns.tolist()) # 이 출력 결과에서 면적 관련 이름을 찾아야 합니다.

#2. 건물면적을 '평수'로 계산한 컬럼 생성
df['평수'] = df['건물면적'] / 3.3058 
print(df['평수'].head().round(2))

# 3. 평형대 구간(Bin) 나누기
# 0~10평, 10~20평, 20~30평, 30~40평, 40평 이상으로 구분합니다.
bins = [0, 10, 20, 30, 40, 1000] # 구간의 경계값을 설정합니다. # (1000은 40평 이상의 모든 값 포함)
labels = ['10평 이하', '10평대', '20평대', '30평대', '40평 이상'] # 각 구간에 붙일 이름표입니다.

# 4. 평형구분 컬럼 생성
df['평형구분'] = pd.cut(df['평수'], bins=bins, labels=labels, right=False) # 평수 데이터를 기준에 따라 구간별로 분류합니다. # (cut 함수 사용)

# 5. 강남 3구 필터링 및 구별/평형별 거래건수 집계
targets = ['강남구', '송파구', '서초구'] # 비교할 대상 구를 리스트로 만듭니다.
comparison_df = df[df['자치구명'].isin(targets)].copy() # 대상 구의 데이터만 복사해옵니다.

# 6. 거래건수 확인 (물건금액 컬럼의 개수를 카운트)
# 자치구명과 평형구분을 기준으로 묶어 건수를 계산합니다.
result = comparison_df.groupby(['자치구명', '평형구분']).agg(
    거래건수=('물건금액', 'count') # 각 그룹별 물건금액의 개수를 세어 거래건수를 구합니다. # (Aggregation)
).reset_index() # 결과표를 보기 좋게 평면 데이터로 펼칩니다.

# 7. 결과 출력
print("[강남 3구 평형대별 거래건수 비교]")
print(result.pivot(index='평형구분', columns='자치구명', values='거래건수')) # 가로축을 구, 세로축을 평형으로 하는 표로 변환합니다.

In [ ]:
import plotly.express as px  # 시각화 라이브러리를 가져옵니다.

# 1. 10평 이하 데이터 제거 (Filtering)
plot_df_filtered = plot_df[plot_df['평형구분'] != '10평 이하'].copy() # '10평 이하' 제외 데이터를 추출합니다.

# 2. 각 자치구 내에서의 비율(%) 계산
total_counts = plot_df_filtered.groupby('자치구명')['거래건수'].transform('sum') # 구별 총합을 구합니다.
plot_df_filtered['비율'] = (plot_df_filtered['거래건수'] / total_counts * 100).round(1) # 백분율을 구합니다.

# 3. 그래프 라벨 생성 (좁은 공간에 맞춰 <br> 태그로 줄바꿈 적용)
plot_df_filtered['라벨'] = plot_df_filtered.apply(
    lambda row: f"{row['비율']}%<br>({int(row['거래건수']):,}건)", axis=1 # 작은 공간에서도 잘 보이도록 줄바꿈합니다.
)

# 4. 누적 막대 그래프 생성 (색상 팔레트 적용)
fig = px.bar(
    plot_df_filtered, # 필터링된 데이터를 사용합니다.
    x='자치구명', # 가로축입니다.
    y='거래건수', # 세로축입니다.
    color='평형구분', # 평형대별 색상 구분입니다.
    text='라벨', # 생성한 라벨을 표시합니다.
    title='<b>강남 3구 평형대별 거래 비중 및 규모</b>', # 제목을 조금 더 간결하게 수정합니다.
    color_discrete_sequence=px.colors.sequential.Blues_r, # 진한 파랑에서 연한 파랑 순서입니다.
    template='plotly_white' # 배경을 흰색으로 설정합니다.
)

# 5. 축소된 사이즈(600x450)에 최적화된 레이아웃 업데이트
fig.update_layout(
    # 제목 사이즈를 전체 비율에 맞춰 조정 (28 -> 20)
    title=dict(font=dict(size=20), x=0.5, y=0.95), 
    
    # 요청하신 슬림한 사이즈 적용
    width=600, # 그래프 너비입니다.
    height=450, # 그래프 높이입니다.
    
    # 좁은 화면에서 막대가 너무 얇아지지 않도록 간격 조정 (0.4 -> 0.3)
    bargap=0.3, 
    
    # X축 텍스트 사이즈 조정 (16 -> 14)
    xaxis=dict(title="", tickfont=dict(size=14)), 
    
    # Y축 숫자 포맷 및 사이즈 조정 (14 -> 12)
    yaxis=dict(
        title="거래건수 (건)", 
        tickformat=",d", 
        tickfont=dict(size=12),
        range=[0, plot_df_filtered.groupby('자치구명')['거래건수'].sum().max() * 1.1] # 상단 여백 확보
    ), 
    
    # 범례(Legend) 사이즈 축소 및 배치 최적화
    legend_title=dict(text="평형대", font=dict(size=12)),
    legend=dict(font=dict(size=10), x=1.02, y=1), # 범례를 살짝 오른쪽으로 밀어 본문 공간 확보
    
    # 여백 최적화 (여백을 줄여 막대 영역을 넓힘)
    margin=dict(l=60, r=20, t=80, b=40)
)

# 막대 내부 텍스트 스타일 및 가독성 최적화
fig.update_traces(
    textposition='inside', # 텍스트를 막대 안에 넣습니다.
    textfont_size=10, # 작은 막대에서도 글자가 깨지지 않도록 사이즈 축소 (12 -> 10)
    textfont_color='white', # 파란색 배경에서 잘 보이도록 흰색 유지
    insidetextanchor='middle' # 텍스트를 막대 칸 중앙에 배치합니다.
)

fig.show() # 최종 결과물을 출력합니다.

In [ ]:
my_chart.show()  

## ==> 박스 플롯으로 강남, 서초에 있을 이상치? 알아보기! 박스플롯으로 했을 때 더 적합할 것 같음 !
## ==> 해석 방법? 을 AI한테 물어보기! 


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# 1. 분석 대상 설정: 2025년 4월 & 강남 3구 데이터 필터링
target_month = '2025-04'
targets = ['강남구', '서초구', '송파구']

# '계약일'에서 연-월 정보를 추출하여 필터링
analysis_df = df[(df['계약일'].dt.to_period('M') == target_month) & 
                 (df['자치구명'].isin(targets))].copy()

# 2. 자치구별 통계 요약 함수 (MECE하게 정보 추출)
def get_box_stats(data, gu_name):
    gu_df = data[data['자치구명'] == gu_name]
    q1 = gu_df['물건금액'].quantile(0.25)
    q3 = gu_df['물건금액'].quantile(0.75)
    iqr = q3 - q1
    upper_fence = q3 + (1.5 * iqr)
    outliers = gu_df[gu_df['물건금액'] > upper_fence]
    
    return {
        '구': gu_name,
        '중앙값': gu_df['물건금액'].median(),
        '평균': gu_df['물건금액'].mean(),
        '이상치경계': upper_fence,
        '이상치건수': len(outliers)
    }

# 각 구별 통계 출력
stats_list = [get_box_stats(analysis_df, gu) for gu in targets]
stats_summary = pd.DataFrame(stats_list)
print(f"--- {target_month} 강남 3구 4분위 통계 비교 ---")
print(stats_summary)

# 3. 박스플롯 시각화 (병렬 비교)
fig = px.box(analysis_df, 
             x="자치구명", 
             y="물건금액", 
             color="자치구명",
             points="all",  # 모든 거래 데이터를 점으로 표시
             title=f"<b>{target_month} 강남 3구 매매가 분포 비교 (Box Plot)</b>",
             category_orders={"자치구명": ["강남구", "서초구", "송파구"]},
             color_discrete_map={'강남구': 'royalblue', '서초구': 'darkorange', '송파구': 'forestgreen'},
             labels={'물건금액':'거래금액(만 원)', '자치구명':''},
             template='plotly_white')

# 디자인 최적화
fig.update_layout(
    showlegend=False,
    width=900, height=600,
    yaxis_tickformat=',d',
    font=dict(size=14)
)

fig.show()

In [ ]:
#분석 구현 코드 (강남 vs 서초 4월 정밀 비교)
#두 구의 데이터 분포를 직접 비교하여 "서초구는 왜 하락했는가"를 보여주는 시각화 코드입니다.

import plotly.express as px

# 1. 2025년 4월 강남구와 서초구 데이터만 추출
target_month = '2025-04'
comparison_df = df[(df['계약일'].dt.to_period('M') == target_month) & 
                    (df['자치구명'].isin(['강남구', '서초구']))].copy()

# 2. 박스플롯 시각화 (서초구의 하락 원인 파악용)
fig = px.box(comparison_df, x="자치구명", y="물건금액", color="자치구명",
             points="all", # 개별 거래 점을 모두 표시하여 거래 분포 확인
             title=f"<b>{target_month} 강남구 vs 서초구 가격 분포 비교</b>",
             color_discrete_map={'강남구': 'royalblue', '서초구': 'darkorange'},
             template='plotly_white')

# 3. 평균값 비교선 추가
for gu in ['강남구', '서초구']:
    avg_val = comparison_df[comparison_df['자치구명'] == gu]['물건금액'].mean()
    fig.add_hline(y=avg_val, line_dash="dot", line_color='gray', 
                  annotation_text=f"{gu} 평균")

fig.show()

# 4. (추가) 거래량 수치 확인
print(comparison_df.groupby('자치구명')['물건금액'].agg(['count', 'mean', 'median']))

In [ ]:
import pandas as pd
import plotly.express as px

# 1. 2025년 4월 강남 3구 데이터 추출
target_month = '2025-04'
targets = ['강남구', '서초구', '송파구']
df_apr = df[(df['계약일'].dt.to_period('M') == target_month) & 
            (df['자치구명'].isin(targets))].copy()

# 2. 동별 집계 (평균가, 거래건수, 최고가)
dong_summary = df_apr.groupby(['자치구명', '법정동명'])['물건금액'].agg(
    평균거래가='mean',
    거래건수='count',
    최고가='max'
).reset_index()

# 3. 버블 차트 시각화 (Price vs Volume Analysis)
# 원의 크기가 클수록 거래 건수가 많음을 의미합니다.
fig = px.scatter(dong_summary, 
                 x="거래건수", 
                 y="평균거래가", 
                 size="거래건수", 
                 color="자치구명",
                 hover_name="법정동명",
                 text="법정동명",
                 title=f"<b>{target_month} 강남 3구 동별 거래가-거래량 분석</b>",
                 labels={'평균거래가':'평균 실거래가 (만 원)', '거래건수':'거래 건수 (건)'},
                 color_discrete_map={'강남구': 'royalblue', '서초구': 'darkorange', '송파구': 'forestgreen'},
                 template='plotly_white',
                 size_max=40)

# 텍스트 위치 및 디자인 조정
fig.update_traces(textposition='top center')
fig.update_layout(
    width=1000, height=700,
    yaxis_tickformat=',d',
    xaxis_title="거래 건수 (Volume)",
    yaxis_title="평균 실거래가 (Price)",
    font=dict(size=12)
)

fig.show()

# 4. 상세 데이터 출력 (Top 10 거래가 순)
print(f"--- {target_month} 강남 3구 동별 상세 현황 ---")
print(dong_summary.sort_values(by='평균거래가', ascending=False).head(15))